In [1]:
import os
from optparse import OptionParser
import sys
import numpy as np
import pandas as pd
import json
import yaml

In [2]:
sys.path.insert(0, '/home/jgutman/mvesc/ETL')
sys.path.insert(0, '/home/jgutman/mvesc/Features')
sys.path.insert(0, '/home/jgutman/mvesc/Models_Results')

In [3]:
!git branch

* add_cross_val3_and_bootstrapping
  adding_allTeachers_to_database
  copy_df2postgres_file_to_utility
  error_analysis
  generate_normalized_test_scores
  gpa_generation
  intervention_dataETL_feature_generation
  master
  mobility_features
  week_update_model_testing


In [4]:
from mvesc_utility_functions import *

In [5]:
from sklearn.cross_validation import *

In [6]:
from estimate_prediction_model import *

In [7]:
model_options = read_in_yaml('/home/jgutman/mvesc/Models_Results/model_options_expand_features.yaml')

In [8]:
model_options['features_included'] = {
    #'demographics': ['ethnicity', 'gender'], 
    'absence': ['absence*', 'absence_unexcused*', 'tardy*', 'tardy_unexcused*', 'medical*', 
                'absence_consec*', 'tardy_consec*']
    #'grades': ['gpa*', 'language_gpa*', 'stem_gpa*', 'humanities_gpa*', 'art_gpa*', 'health_gpa*', 
    #    'future_prep_gpa*', 'interventions_gpa*', 'language_gpa*', 'num_language_classes*', 'num_stem_classes*',
    #    'num_humanities_classes*', 'num_art_classes*', 'num_health_classes*', 'num_future_prep_classes*',
    #    'num_interventions_classes*', 'num_language_classes*', 'percent_passed_pf_classes*', 'num_pf_classes*',
    #    'gpa_district*'], 
    #'mobility': ['n_addresses_to*', 'n_districts_to*', 'n_cities_to*', 'n_records_to*', 'avg_address_change_to*',
    #    'avg_district_change_to*', 'avg_city_change_to*', 'street_transition_in*', 'district_transition_in*',
    #    'city_transition_in*', 'mid_year_withdraw*'],
    #'snapshots': ['disadvantagement*', 'disability*', 'gifted*', 'iss*', 'oss*', 'limited_english*', 
    #    'special_ed*', 'discipline_incidents*']
}

In [9]:
outcome_plus_features = build_outcomes_plus_features(model_options)

In [10]:
outcome_plus_features.dropna(subset=[model_options['outcome_name'],
        model_options['cohort_grade_level_begin']], inplace=True)

In [11]:
train, test = temporal_cohort_test_split(outcome_plus_features,
            model_options['cohort_grade_level_begin'],
            model_options['cohorts_held_out'],
            model_options['cohorts_training'])

In [12]:
print(train.shape)
print(test.shape)

(2127, 30)
(2009, 30)


In [13]:
train_X = train.drop([model_options['outcome_name'],
        model_options['cohort_grade_level_begin']],axis=1)
test_X = test.drop([model_options['outcome_name'],
        model_options['cohort_grade_level_begin']],axis=1)
train_y = train[model_options['outcome_name']]
test_y = test[model_options['outcome_name']]

In [14]:
# do missing value feature imputation here
train_X, test_X = impute_missing_values(train_X, test_X,
    model_options['missing_impute_strategy'])
assert (all(train_X.columns == test_X.columns)),\
    "train and test have different columns"

/home/jgutman/env/lib/python3.4/site-packages/sklearn/preprocessing/imputation.py:344: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 56 but corresponding boolean dimension is 28
  missing = np.arange(X.shape[not self.axis])[invalid_mask]


ValueError: operands could not be broadcast together with shape (28,) (56,)

In [15]:
train_X.isnull().sum()

absence_consec_gr_8       0
tardy_gr_7                0
tardy_gr_8                0
tardy_unexcused_gr_6      0
tardy_consec_gr_9         0
absence_unexcused_gr_9    0
medical_gr_6              0
absence_gr_7              0
tardy_unexcused_gr_8      0
tardy_consec_gr_7         0
absence_gr_6              0
absence_gr_8              0
tardy_unexcused_gr_7      0
tardy_gr_9                0
absence_unexcused_gr_8    0
tardy_gr_6                0
tardy_unexcused_gr_9      0
absence_unexcused_gr_7    0
absence_consec_gr_6       0
medical_gr_9              0
tardy_consec_gr_6         0
absence_unexcused_gr_6    0
tardy_consec_gr_8         0
medical_gr_7              0
absence_consec_gr_9       0
absence_consec_gr_7       0
absence_gr_9              0
medical_gr_8              0
dtype: int64

In [16]:
imputer = Imputer(strategy='median')

In [17]:
imputer.fit(train_X)

Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)

In [20]:
imputer.transform(train_X).shape

(2127, 28)

In [21]:
train_X.shape

(2127, 28)

In [24]:
pd.DataFrame(imputer.transform(train_X), # returns a numpy array
            columns = train_X.columns, index = train_X.index)

,absence_consec_gr_8,tardy_gr_7,tardy_gr_8,tardy_unexcused_gr_6,tardy_consec_gr_9,absence_unexcused_gr_9,medical_gr_6,absence_gr_7,tardy_unexcused_gr_8,tardy_consec_gr_7,...,absence_consec_gr_6,medical_gr_9,tardy_consec_gr_6,absence_unexcused_gr_6,tardy_consec_gr_8,medical_gr_7,absence_consec_gr_9,absence_consec_gr_7,absence_gr_9,medical_gr_8
student_lookup,,,,,,,,,,,,,,,,,,,,,
37505.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,0.0
51059.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
34646.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0
41514.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.5,0.0
41513.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.5,0.0
41512.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,5.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
51416.0,0.0,0.0,0.0,0.0,0.0,7.5,0.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,22.0,0.0
41511.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0
41510.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0


In [27]:
add_null_dummies(train_X).shape

(2127, 28)

In [28]:
train_X.shape

(2127, 28)

In [29]:
impute_missing_values(train_X, test_X, 'median_plus_dummies')

/home/jgutman/env/lib/python3.4/site-packages/sklearn/preprocessing/imputation.py:344: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 56 but corresponding boolean dimension is 28
  missing = np.arange(X.shape[not self.axis])[invalid_mask]


ValueError: operands could not be broadcast together with shape (28,) (56,)

In [30]:
train = add_null_dummies(train_X)
test = add_null_dummies(test_X)

In [32]:
train.shape

(2127, 28)

In [33]:
train_X.shape

(2127, 28)

In [34]:
test_X.shape

(2009, 28)

In [35]:
test.shape

(2009, 56)

In [36]:
test.columns

Index(['absence_consec_gr_8', 'tardy_gr_7', 'tardy_gr_8',
       'tardy_unexcused_gr_6', 'tardy_consec_gr_9', 'absence_unexcused_gr_9',
       'medical_gr_6', 'absence_gr_7', 'tardy_unexcused_gr_8',
       'tardy_consec_gr_7', 'absence_gr_6', 'absence_gr_8',
       'tardy_unexcused_gr_7', 'tardy_gr_9', 'absence_unexcused_gr_8',
       'tardy_gr_6', 'tardy_unexcused_gr_9', 'absence_unexcused_gr_7',
       'absence_consec_gr_6', 'medical_gr_9', 'tardy_consec_gr_6',
       'absence_unexcused_gr_6', 'tardy_consec_gr_8', 'medical_gr_7',
       'absence_consec_gr_9', 'absence_consec_gr_7', 'absence_gr_9',
       'medical_gr_8', 'absence_consec_gr_8_isnull', 'tardy_gr_7_isnull',
       'tardy_gr_8_isnull', 'tardy_unexcused_gr_6_isnull',
       'tardy_consec_gr_9_isnull', 'absence_unexcused_gr_9_isnull',
       'medical_gr_6_isnull', 'absence_gr_7_isnull',
       'tardy_unexcused_gr_8_isnull', 'tardy_consec_gr_7_isnull',
       'absence_gr_6_isnull', 'absence_gr_8_isnull',
       'tardy_unexcu

In [38]:
data = test_X

In [39]:
data.shape

(2009, 28)

In [40]:
data[data.columns[data.isnull().sum() > 0]]

,absence_consec_gr_8,tardy_gr_7,tardy_gr_8,tardy_unexcused_gr_6,tardy_consec_gr_9,absence_unexcused_gr_9,medical_gr_6,absence_gr_7,tardy_unexcused_gr_8,tardy_consec_gr_7,...,absence_consec_gr_6,medical_gr_9,tardy_consec_gr_6,absence_unexcused_gr_6,tardy_consec_gr_8,medical_gr_7,absence_consec_gr_9,absence_consec_gr_7,absence_gr_9,medical_gr_8
student_lookup,,,,,,,,,,,,,,,,,,,,,
48.0,7.0,0.0,2.0,0.0,0.0,2.0,0.0,6.5,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,9.00,0.0
297.0,17.0,10.0,5.0,0.0,2.0,11.0,0.0,27.5,5.0,3.0,...,0.0,0.0,0.0,2.0,4.0,0.0,5.0,20.0,18.50,0.0
558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
559.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
616.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
649.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.00,0.0
691.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
705.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98,0.0


In [41]:
train_X.isnull().sum()

absence_consec_gr_8       0
tardy_gr_7                0
tardy_gr_8                0
tardy_unexcused_gr_6      0
tardy_consec_gr_9         0
absence_unexcused_gr_9    0
medical_gr_6              0
absence_gr_7              0
tardy_unexcused_gr_8      0
tardy_consec_gr_7         0
absence_gr_6              0
absence_gr_8              0
tardy_unexcused_gr_7      0
tardy_gr_9                0
absence_unexcused_gr_8    0
tardy_gr_6                0
tardy_unexcused_gr_9      0
absence_unexcused_gr_7    0
absence_consec_gr_6       0
medical_gr_9              0
tardy_consec_gr_6         0
absence_unexcused_gr_6    0
tardy_consec_gr_8         0
medical_gr_7              0
absence_consec_gr_9       0
absence_consec_gr_7       0
absence_gr_9              0
medical_gr_8              0
dtype: int64

In [43]:
test_X

,absence_consec_gr_8,tardy_gr_7,tardy_gr_8,tardy_unexcused_gr_6,tardy_consec_gr_9,absence_unexcused_gr_9,medical_gr_6,absence_gr_7,tardy_unexcused_gr_8,tardy_consec_gr_7,...,absence_consec_gr_6,medical_gr_9,tardy_consec_gr_6,absence_unexcused_gr_6,tardy_consec_gr_8,medical_gr_7,absence_consec_gr_9,absence_consec_gr_7,absence_gr_9,medical_gr_8
student_lookup,,,,,,,,,,,,,,,,,,,,,
48.0,7.0,0.0,2.0,0.0,0.0,2.0,0.0,6.5,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,4.0,9.00,0.0
297.0,17.0,10.0,5.0,0.0,2.0,11.0,0.0,27.5,5.0,3.0,...,0.0,0.0,0.0,2.0,4.0,0.0,5.0,20.0,18.50,0.0
558.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
559.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
616.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
649.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.00,0.0
691.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
705.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
706.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.98,0.0


In [49]:
train_X.columns[train_X.isnull().sum() > 0]

Index([], dtype='object')

In [52]:
set(train_X.columns[train_X.isnull().sum() > 0]).add(test_X.columns[test_X.isnull().sum() > 0])

TypeError: unhashable type: 'Index'

In [70]:
train_null_columns = train_X.columns[train_X.isnull().sum() > 0]
test_null_columns = test_X.columns[test_X.isnull().sum() > 0]
null_columns = pd.indexes.base.Index(set(train_null_columns).union(set(test_null_columns)))

In [63]:
train = train_X.copy()
test = test_X.copy()

In [71]:
for data in [train, test]:
    data_null_columns = data[null_columns]
    data_null_dummies = data_null_columns.isnull()*1.0
    data_null_dummies.rename(columns=lambda x: x + '_isnull', inplace=True)
    data = data.merge(data_null_dummies,
        left_index=True, right_index=True)

In [73]:
test.shape

(2009, 28)

In [74]:
null_columns

Index(['tardy_gr_9', 'absence_unexcused_gr_8', 'tardy_gr_6',
       'absence_consec_gr_8', 'tardy_unexcused_gr_9', 'tardy_gr_7',
       'tardy_gr_8', 'absence_unexcused_gr_7', 'tardy_unexcused_gr_6',
       'absence_consec_gr_6', 'tardy_consec_gr_9', 'medical_gr_9',
       'tardy_consec_gr_6', 'absence_unexcused_gr_9', 'medical_gr_6',
       'absence_gr_7', 'absence_unexcused_gr_6', 'tardy_consec_gr_8',
       'tardy_unexcused_gr_8', 'tardy_consec_gr_7', 'medical_gr_7',
       'absence_gr_6', 'absence_consec_gr_9', 'absence_consec_gr_7',
       'absence_gr_8', 'absence_gr_9', 'tardy_unexcused_gr_7', 'medical_gr_8'],
      dtype='object')

In [83]:
def add_null_dummies_train_test(train, test):
    """
    """
    train_null_columns = train.columns[train.isnull().sum() > 0]
    test_null_columns = test.columns[test.isnull().sum() > 0]
    null_column_set = set(train_null_columns).union(set(test_null_columns))
    null_columns = pd.indexes.base.Index(null_column_set)

    train_nullified = add_null_dummies(train, null_columns)
    test_nullified = add_null_dummies(test, null_columns)
    return train_nullified, test_nullified

def add_null_dummies(data, null_columns):
    data_null_columns = data[null_columns]
    data_null_dummies = data_null_columns.isnull()*1.0
    data_null_dummies.rename(columns=lambda x: x + '_isnull', inplace=True)
    data_plus_dummies = data.merge(data_null_dummies,
        left_index=True, right_index=True)
    return data_plus_dummies

In [84]:
train2, test2 = add_null_dummies_train_test(train=train_X, test=test_X)

<class 'pandas.indexes.base.Index'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [85]:
train2.shape

(2127, 56)

In [86]:
test2.shape

(2009, 56)